In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

import torch
from torch import Tensor
import torch.nn as nn
import torch.optim as optim
print(torch.__version__)

2.6.0+cu124


In [54]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

In [55]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [56]:
df.shape

(569, 33)

In [57]:
df = df.drop(['id','Unnamed: 32'],axis = 1)

In [58]:
df.shape

(569, 31)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [60]:
X = df.drop(['diagnosis'], axis = 1)
y = df['diagnosis']

In [61]:
X_train,X_test, y_train,y_test=train_test_split(X,y,test_size=0.2,random_state = 43)

print('X_train shape = ',X_train.shape)
print('X_test shape = ',X_test.shape)
print('y_train shape = ',y_train.shape)
print('y_test shape = ',y_test.shape)

X_train shape =  (455, 30)
X_test shape =  (114, 30)
y_train shape =  (455,)
y_test shape =  (114,)


In [62]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [63]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [64]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [65]:
y_train_tensor = (y_train_tensor).view(-1,1).to(torch.float64)

In [66]:
class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1,dtype=torch.float64)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [67]:
learning_rate = 0.1
epochs = 25

# define loss function
loss_function = nn.BCELoss()

In [68]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor.view(-1,1))

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.6321371041364373
Epoch: 2, Loss: 0.4933999617941225
Epoch: 3, Loss: 0.41708572450812653
Epoch: 4, Loss: 0.36815736459220505
Epoch: 5, Loss: 0.3336092772531046
Epoch: 6, Loss: 0.3076337246610507
Epoch: 7, Loss: 0.2872266547496017
Epoch: 8, Loss: 0.2706666095719024
Epoch: 9, Loss: 0.25689061637846794
Epoch: 10, Loss: 0.2452038063180335
Epoch: 11, Loss: 0.23513100513622429
Epoch: 12, Loss: 0.22633516802179235
Epoch: 13, Loss: 0.21856987294379177
Epoch: 14, Loss: 0.21165030204724056
Epoch: 15, Loss: 0.2054347953230688
Epoch: 16, Loss: 0.19981271994429853
Epoch: 17, Loss: 0.1946962557566676
Epoch: 18, Loss: 0.1900146886460962
Epoch: 19, Loss: 0.1857103558963931
Epoch: 20, Loss: 0.1817357072117105
Epoch: 21, Loss: 0.17805113610663845
Epoch: 22, Loss: 0.17462335392311198
Epoch: 23, Loss: 0.17142415297471214
Epoch: 24, Loss: 0.16842945331055598
Epoch: 25, Loss: 0.16561855927512162


In [69]:
model.linear.weight

Parameter containing:
tensor([[ 0.3856,  0.1730,  0.2408,  0.1327,  0.1013, -0.0544,  0.2546,  0.1596,
          0.1901,  0.0057,  0.3103,  0.0236,  0.0617,  0.0958,  0.0052, -0.0719,
          0.0603,  0.1403, -0.0424,  0.0657,  0.3663,  0.2016,  0.0986,  0.4181,
          0.2122,  0.2142,  0.0350,  0.2717,  0.2766,  0.0318]],
       dtype=torch.float64, requires_grad=True)

In [70]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.549861490726471
